In [3]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-06-04 13:46:17--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  6.07MB/s    in 0.2s    

2025-06-04 13:46:18 (6.07 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [4]:
with open('input.txt', 'r', encoding='UTF-8') as f:
    text = f.read()

In [7]:
print('Length of dataset in characters:', len(text))

Length of dataset in characters: 1115394


In [13]:
# First 1000 characters
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [10]:
# All unique characters which appear in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
# We now with to tokenize characters to a sequence of integers
# Possible strategies include direct mappings for each character to some integer
# which leads to the smallest vocabulary, but largest expression
# vs. a sub-word level tokenizer which a much larger vocabulary but with shorter expressions
# Other schemas include tiktoken (Used by gpt) or SentencePiece (google)

In [11]:
# Character level tokenizer, direct mapping from character to integer
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[ch] for ch in s] # takes a string and encodes into a list of integers
decode = lambda l: ''.join([itos[i] for i in l])

print(encode('Hello World'))
print(decode(encode('Hello World')))

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42]
Hello World


In [15]:
# lets now encode the entire dataset and store it in a tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [17]:
# Lets now split up the data into training and validation sets
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
# we never feed the entire text to the transformer at any given time. This
# would be computationally expensive and prohibitive. Instead we break the text
# into smaller chunks with some maximum length (Block size / Context length)

In [18]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
# We break up the sample in this way so the model
# can get used to seeing contexts of varying length
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'When input is {context} the target: {target}')

When input is tensor([18]) the target: 47
When input is tensor([18, 47]) the target: 56
When input is tensor([18, 47, 56]) the target: 57
When input is tensor([18, 47, 56, 57]) the target: 58
When input is tensor([18, 47, 56, 57, 58]) the target: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [25]:
torch.manual_seed(1337)
batch_size = 4 # independent sequences that will be processed in parallel
block_size = 8 # maximum context length

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    # generates random integers up to len(data) - block size to ensure there is
    # enough space to get full context, of dimension (batch_size, )
    # i.e. generates 4 random indexes
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    # grab full context list for each random index
    # and combines each 1D tensor into a 2D tensor
    # with torch.stack() (which also does not use new memory)
    x = torch.stack([data[i:i+block_size] for i in ix])
    # grab all targets for each context list
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b,:t+1]
        target = yb[b, t]
        print(f'When input is {context}, the target: {target}')


inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
When input is tensor([24]), the target: 43
When input is tensor([24, 43]), the target: 58
When input is tensor([24, 43, 58]), the target: 5
When input is tensor([24, 43, 58,  5]), the target: 57
When input is tensor([24, 43, 58,  5, 57]), the target: 1
When input is tensor([24, 43, 58,  5, 57,  1]), the target: 46
When input is tensor([24, 43, 58,  5, 57,  1, 46]), the target: 43
When input is tensor([24, 43, 58,  5, 57,  1, 46, 43]), the target: 39
When input is tensor([44]), the target: 53
When input is tensor([44, 53]), the target: 56
When input is tensor([44, 53, 56]), the target: 1
When inp

In [ ]:
import torch
import torch.nn as nn 
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets):
        logits = self.token_embedding_table(idx)
        return logits
